<a href="https://colab.research.google.com/github/Tim-Ganther/DeepLearning-NameToGender/blob/master/Name2Gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [0]:
fnames = pd.read_csv("female-names.csv", header=None, dtype={0: str})
mnames = pd.read_csv("male-names.csv", header=None, dtype={0: str})
fnames = fnames.rename(columns={0:"Name"})
mnames = mnames.rename(columns={0:"Name"})
#fnames["Geschlecht"] = 0
#mnames["Geschlecht"] = 1
#namen = pd.concat([fnames, mnames])
#namen = namen.reset_index(drop=True)

In [0]:
mnames["Name"] = mnames.Name.str.replace("ü", "ue" , regex= True)
mnames["Name"] = mnames.Name.str.replace("ö", "oe" , regex= True)
mnames["Name"] = mnames.Name.str.replace("ä", "ae" , regex= True)
fnames["Name"] = fnames.Name.str.replace("ü", "ue" , regex= True)
fnames["Name"] = fnames.Name.str.replace("ö", "oe" , regex= True)
fnames["Name"] = fnames.Name.str.replace("ä", "ae" , regex= True)

mnames["Name"] = mnames.Name.str.replace("Ü", "Ue" , regex= True)
mnames["Name"] = mnames.Name.str.replace("Ö", "Oe" , regex= True)
mnames["Name"] = mnames.Name.str.replace("Ä", "Ae" , regex= True)
fnames["Name"] = fnames.Name.str.replace("Ü", "Ue" , regex= True)
fnames["Name"] = fnames.Name.str.replace("Ö", "Oe" , regex= True)
fnames["Name"] = fnames.Name.str.replace("Ä", "Ae" , regex= True)

mnames["Name"] = mnames.Name.str.replace("é", "e" , regex= True)
fnames["Name"] = fnames.Name.str.replace("é", "e" , regex= True)

mnames["Name"] = mnames.drop(mnames.index[mnames.Name.str.contains("-", regex= True, na=False)].tolist())
fnames["Name"] = fnames.drop(fnames.index[fnames.Name.str.contains("-", regex= True, na=False)].tolist())
fnames = fnames.dropna()
mnames = mnames.dropna()

fnames = fnames.reset_index(drop=True)
mnames = mnames.reset_index(drop=True)

In [0]:
gender = []
gender.append('f')
gender.append('m')
name_list = {}
name_list['f'] = fnames.values.tolist()
name_list['m'] = mnames.values.tolist()

In [0]:
import string
all_letters = string.ascii_letters
n_letters = len(all_letters)

def letterToIndex(letter):
    return all_letters.find(letter)

def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(lineToTensor('Lenny'))

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.]],

        [[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.]

In [0]:
import torch.nn as nn

n_categories = len(gender)

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [0]:
input = lineToTensor('Tim')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
#print(output)

def genderFromOutput(output):
    top_n, top_i = output.topk(1)
    gender_i = top_i[0].item()
    return gender[gender_i], gender_i

print(genderFromOutput(output))

('f', 0)


In [0]:
import random

def randomChoice(l):
    h = random.randint(0, len(l) - 1)
    return l[h]

def randomTrainingExample():
    rdm_gender = randomChoice(gender)
    line = randomChoice(name_list[rdm_gender])[0]
    gender_tensor = torch.tensor([gender.index(rdm_gender)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return rdm_gender, line, gender_tensor, line_tensor

for i in range(10):
    rdm_gender, line, gender_tensor, line_tensor = randomTrainingExample()
    print('gender =', rdm_gender, '/ line =', line)

gender = f / line = Hertha
gender = f / line = Ortrud
gender = f / line = Mathilde
gender = m / line = Winrich
gender = m / line = Malte
gender = m / line = Alfons
gender = m / line = Roberto
gender = f / line = Irmelie
gender = m / line = Albrecht
gender = f / line = Adelheid


In [0]:
randomChoice(name_list[randomChoice(gender)])[0]

'Gunfried'

In [0]:
criterion = nn.NLLLoss()
learning_rate = 0.005 

def train(gender_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, gender_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.__add__(-learning_rate, p.grad)

    return output, loss.item()

In [0]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    rdm_gender, line, gender_tensor, line_tensor = randomTrainingExample()
    output, loss = train(gender_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = genderFromOutput(output)
        correct = '✓' if guess == rdm_gender else '✗ (%s)' % rdm_gender
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

5000 5% (0m 7s) 0.7579 Eike / f ✗ (m)
10000 10% (0m 15s) 0.6608 Katja / f ✓
15000 15% (0m 23s) 0.6491 Edwine / f ✓
20000 20% (0m 30s) 0.7611 Altrud / m ✗ (f)
25000 25% (0m 38s) 0.6650 Egbert / m ✓
30000 30% (0m 45s) 0.6381 Oslinde / f ✓
35000 35% (0m 53s) 0.7567 Gordian / f ✗ (m)
40000 40% (1m 1s) 0.6398 Cathrin / f ✓
45000 45% (1m 8s) 0.6800 Brandolf / m ✓
50000 50% (1m 16s) 0.6523 Godo / m ✓
55000 55% (1m 23s) 0.7420 Marzel / f ✗ (m)
60000 60% (1m 31s) 0.6497 Haimo / m ✓
65000 65% (1m 38s) 0.7036 Jonas / f ✗ (m)
70000 70% (1m 46s) 0.6422 Volrad / m ✓
75000 75% (1m 53s) 0.6740 Kreszenz / f ✓
80000 80% (2m 1s) 0.6498 Baerbel / f ✓
85000 85% (2m 8s) 0.6354 Hanneliese / f ✓
90000 90% (2m 16s) 0.7420 Marzel / f ✗ (m)
95000 95% (2m 23s) 0.7235 Diemut / m ✗ (f)
100000 100% (2m 31s) 0.7571 Luitwin / f ✗ (m)
